In [28]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn import tree
from sklearn import metrics
from sklearn import preprocessing 
from sklearn.ensemble import RandomForestClassifier
import json
from sklearn.feature_extraction.text import CountVectorizer


In [29]:
df_test=pd.read_json('/content/test.json')
df_train=pd.read_json('/content/train.json')

In [30]:
df_train.head()

,id,cuisine,ingredients
0,10259,greek,"[romaine lettuce, black olives, grape tomatoes..."
1,25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g..."
2,20130,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,22213,indian,"[water, vegetable oil, wheat, salt]"
4,13162,indian,"[black pepper, shallots, cornflour, cayenne pe..."


In [31]:
ingredients_train = df_train.ingredients
words_train = [' '.join(x) for x in ingredients_train]
print(len(words_train), words_train[0])

39774 romaine lettuce black olives grape tomatoes garlic pepper purple onion seasoning garbanzo beans feta cheese crumbles


In [32]:
ingredients_test = df_test.ingredients
words_test = [' '.join(x) for x in ingredients_test]

In [33]:
vectorizer = CountVectorizer(max_features = 1000)
bag_of_words = vectorizer.fit(words_train)
bag_of_words

CountVectorizer(max_features=1000)

In [34]:
ing_array_train = bag_of_words.transform(words_train).toarray()
ing_array_test = bag_of_words.transform(words_test).toarray()
ing_array_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [35]:
df_ing_train = pd.DataFrame(ing_array_train, columns=vectorizer.vocabulary_)
df_ing_test = pd.DataFrame(ing_array_test, columns=vectorizer.vocabulary_)
df_ing_train.head()

,romaine,lettuce,black,olives,grape,tomatoes,garlic,pepper,purple,onion,...,liver,muffin,stick,chow,fusilli,fronds,flavor,fajita,chayotes,american
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [36]:
df_train_new = df_train.merge(df_ing_train, 
                          left_index=True, 
                          right_index=True).drop('ingredients', axis=1)
df_train_new.head()

,id,cuisine,romaine,lettuce,black,olives,grape,tomatoes,garlic,pepper,...,liver,muffin,stick,chow,fusilli,fronds,flavor,fajita,chayotes,american
0,10259,greek,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,25693,southern_us,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,20130,filipino,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,22213,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,13162,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [37]:
df_test_new = df_test.merge(df_ing_test, 
                          left_index=True, 
                          right_index=True).drop('ingredients', axis=1)
df_test_new.head()

,id,romaine,lettuce,black,olives,grape,tomatoes,garlic,pepper,purple,...,liver,muffin,stick,chow,fusilli,fronds,flavor,fajita,chayotes,american
0,18009,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,28583,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,41580,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,29752,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
4,35687,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
X = df_train_new.drop(['id', 'cuisine'], axis=1)
y = df_train_new.cuisine
X.shape, y.shape

((39774, 1000), (39774,))

In [39]:
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.85)
X_train.shape, X_val.shape

((33807, 1000), (5967, 1000))

In [40]:
from sklearn.neighbors import KNeighborsClassifier

In [41]:
my_RandomForest = RandomForestClassifier(n_estimators=19, bootstrap=True, random_state=3)
my_RandomForest.fit(X_train, y_train)

my_knn = KNeighborsClassifier(n_neighbors=5)
my_knn.fit(X_train, y_train)

KNeighborsClassifier()

In [42]:
y_predict_knn = my_knn.predict(X_val)
my_rf_predict = my_RandomForest.predict(X_val)

knn_accuracy = accuracy_score(y_val, y_predict_knn)
rf_accuracy = accuracy_score(y_val, my_rf_predict)

print("random forrest accuracy", rf_accuracy)
print("knn accuracy", knn_accuracy)

random forrest accuracy 0.7415786827551534
knn accuracy 0.6363331657449305
